# From OMOP to IDEA4RC Surgery

In [ ]:
# Conexión a la base de datos OMOP CDM
conn = psycopg2.connect(
    dbname="omopdb",
    user="postgres",
    password="mysecretpassword",
    host="localhost",
    port="5432"
)

# Comando para hacer el mapeo (insercción de datos del IDEA4RC.csv a OMOP)
curOMOP = conn.cursor()
config = {
    'user': 'user', 
    'password': 'password',
    'host': '127.0.0.1',
    'database': 'idea4rc_dm',
    'raise_on_warnings': True
}

conn2 = mysql.connector.connect(**config)
curIDEA = conn2.cursor()

### From ProcedureOccurrence to IDEA

I do not know how to get the H&N stage since we do not have any concept id to help us and it is not clear which one in Athena would be the correct one. Same with Metastasis

TO BE DONE

In [ ]:
selectEpisodes = "SELECT DISTINCT episode_id, episode_start_date FROM episode_event"
curOMOP.execute(selectEpisodes)
episodes = curOMOP.fetchall()
selectProcedureOccurrence = """SELECT procedure_date, procedure_concept_id, procedure_occurrence_id FROM procedure_occurrence INNER JOIN episode_event ON procedure_occurrence.visit_occurrence_id = episode_event.event_id
WHERE episode_id = %s AND procedure_concept_id IN (4301351,4315400,4217482,4181193)"""
episodeIDEA="""
    SELECT id FROM cancer_episode WHERE patient = %s AND start_date = %s"""
insert= """INSERT INTO surgery (episode_event, procedure_concept_id, visit_occurrence_id) VALUES (%s, %s, %s) RETURNING id"""
occurrencesList={}
for episode in episodes:
    curIDEA.execute(episodeIDEA, (episode[0], episode[1]))
    episode_id = curIDEA.fetchone()
    curOMOP.execute(selectProcedureOccurrence, (episode_id[0]))
    procedures = curOMOP.fetchall()
    for procedure in procedures:
        curIDEA.execute(insert, (episode_id[0], procedure[1], procedure[2]))  
        id = curIDEA.fetchone()  
        occurrencesList[id[0]] = procedure[2]   # To avoid duplicate code and excessive queries, we will store the procedure_occurrence_id in a list and use it for the measurements

        #TODO
    






### From Measurement to IDEA
Here we find a similar problem, since we need to find the way to 

Surgical specimen grading only in untreated tumours, Surgical specimen Mitotic count, Margins after surgery, Type of surgical approach on Tumour, Surgery intention?

Assuming measurement_event_id is the procedure occurrence.

In [ ]:
updt="""
    UPDATE surgery SET surgery_id = %s WHERE procedure_occurrence_id = %s"""

columnsMeasurement= {
    36768904: "Tumor rupture",
    3010988: "Reconstruction",
    35225724: "Site of surgery on metastasis_soft tissue",
    36769243: "Site of surgery on metastasis_distant lymph nodes",
    36770283: "Site of surgery on metastasis_lung",
    36769301: "Site of surgery on metastasis_bone",
    36770544: "Site of surgery on metastasis_liver",
    35226258: "Site of surgery on metastasis_pleura",
    35226253: "Site of surgery on metastasis_peritoneum",
    36768862: "Site of surgery on metastasis_brain",
    36769180: "Site of surgery on metastasis_other viscera",
    4129922: "Site of surgery on metastasis_unknown",
    4137797: "Surgical complications (Clavien-Dindo Classification)"
}
selectMeas="""SELECT measurement_concept_id, value_as_concept_id FROM measurement WHERE measurement_event_id = %s AND measurement_concept_id = %s"""
booleans=['Tumor rupture','Reconstruction','Neck surgery','Surgery on M',
             "Site of surgery on metastasis_soft tissue",
             "Site of surgery on metastasis_distant lymph nodes","Site of surgery on metastasis_lung",
             "Site of surgery on metastasis_bone","Site of surgery on metastasis_liver",
             "Site of surgery on metastasis_pleura","Site of surgery on metastasis_perimeasurement_event_idtoneum",
             "Site of surgery on metastasis_brain","Site of surgery on metastasis_other viscera","Site of surgery on metastasis_unknown"]
for surgery in occurrencesList.keys():
    for col in columnsMeasurement.keys():
        select = """SELECT measurement_id FROM measurement WHERE procedure_occurrence_id = %s AND measurement_concept_id = %s"""
        curOMOP.execute(selectMeas, (occurrencesList[surgery], col))
        measurement = curOMOP.fetchone()
        if measurement:
            columnName=columnsMeasurement.get(col)
            update = """UPDATE measurement SET {columnName} = %s WHERE id = %s"""
            curIDEA.execute(select, (surgery, col))
            if columnName in booleans:
                curIDEA.execute(update, (1,surgery ))            
            else:
                curIDEA.execute(update, (measurement[1],surgery ))
        elif columnsMeasurement.get(col) in booleans:
            columnName=columnsMeasurement.get(col)
            update = """UPDATE measurement SET {columnName} = %s WHERE id = %s"""
            curIDEA.execute(update, (0,surgery ))